# Tensorflow文件的读取

In [1]:
import tensorflow as tf
import os

模拟一下同步先处理数据，然后才能取出数据

In [2]:
# 首先定义队列
Q=tf.FIFOQueue(3,tf.float32)

# 放入一些数据
enq_many=Q.enqueue_many([[0.1,0.2,0.3],])

In [3]:
# 定义取数据的过程
out_q=Q.dequeue()
data=out_q+1
en_q=Q.enqueue(data)

In [6]:
with tf.Session() as sess:
    # 初始化队列
    sess.run(enq_many)
    # 处理数据
    for i in range(100):
        sess.run(en_q)
    # 训练数据
    for i in range(Q.size().eval()):
         print(sess.run(Q.dequeue()))

33.2
33.3
34.1


模拟异步，子线程存入样本，主线程，读取样本

In [2]:
# 定义一个队列，1000
Q=tf.FIFOQueue(1000,tf.float32)

In [3]:
# 定义子线程要做的事情：循环，加一，放入队列
var=tf.Variable(0.0)
# 实现自增
data=tf.assign_add(var,tf.constant(1.0))
en_q=Q.enqueue(data)

In [4]:
# 定义队列管理器op，定义多少个子线程，子线程该干的事情
qr=tf.train.QueueRunner(Q,enqueue_ops=[en_q]*2)

In [5]:
# 初始化变量op
init_op=tf.global_variables_initializer()

In [6]:
with tf.Session() as sess:
    # 初始化变量
    sess.run(init_op)

    # 开启线程管理器
    coord=tf.train.Coordinator()

    # 真正开启子线程
    threads=qr.create_threads(sess,coord=coord,start=True)

    # 主线程，不断读取数据
    for i in range(300):
        print(sess.run(Q.dequeue()))

    # 回收子线程
    coord.request_stop()
    coord.join(threads)

8.0
16.0
28.0
36.0
44.0
53.0
61.0
68.0
74.0
77.0
85.0
95.0
104.0
114.0
125.0
132.0
141.0
148.0
158.0
165.0
174.0
185.0
194.0
203.0
212.0
219.0
226.0
233.0
239.0
247.0
256.0
260.0
268.0
274.0
286.0
292.0
299.0
305.0
311.0
320.0
326.0
337.0
346.0
354.0
362.0
368.0
375.0
385.0
398.0
406.0
413.0
420.0
426.0
434.0
442.0
450.0
460.0
467.0
475.0
483.0
489.0
497.0
504.0
514.0
522.0
532.0
546.0
559.0
568.0
578.0
587.0
595.0
602.0
612.0
622.0
627.0
635.0
642.0
648.0
656.0
664.0
670.0
679.0
684.0
691.0
698.0
704.0
712.0
722.0
729.0
736.0
742.0
748.0
760.0
767.0
773.0
781.0
790.0
797.0
803.0
814.0
821.0
840.0
852.0
862.0
871.0
882.0
890.0
897.0
907.0
913.0
921.0
929.0
938.0
943.0
951.0
957.0
966.0
973.0
980.0
989.0
995.0
1003.0
1010.0
1022.0
1033.0
1042.0
1050.0
1058.0
1067.0
1075.0
1085.0
1093.0
1098.0
1110.0
1118.0
1125.0
1131.0
1140.0
1141.0
1142.0
1143.0
1144.0
1145.0
1146.0
1147.0
1148.0
1149.0
1150.0
1151.0
1152.0
1153.0
1154.0
1155.0
1156.0
1157.0
1158.0
1159.0
1160.0
1161.0
1162.0
1163.0
1

## 文件读取API

In [1]:
import tensorflow as tf
import os

In [2]:
# 找到文件，放入列表,放入列表
file_name=os.listdir('./Data/day_2/')
flielist=[os.path.join('./Data/day_2/',file) for file in file_name]
print(flielist)

['./Data/day_2/sample_submission.csv', './Data/day_2/test.csv', './Data/day_2/train.csv']


In [3]:
# 1、构造文件队列
file_queue=tf.train.string_input_producer(flielist)

In [4]:
# 2、构造阅读器读取数据（按一行）
reader=tf.TextLineReader()
key,value=reader.read(file_queue)
print(value)

Tensor("ReaderReadV2:1", shape=(), dtype=string)


In [5]:
# 3、对每行内容进行解码
# record_defaults:指定每一个样本的每一列的类型，指定默认值
records=[['None'],['None']]
example,label=tf.decode_csv(value,record_defaults=records)
# print(example,label)

In [6]:
# 4、读取多个数据，批处理
# batch_size决定每次取多少数据，和capacity大小没有关系
# batch_size超过样本容量也没关系，超过容量后，从头开始取
# 一般设置batch_size=capacity

example_batch,label_batch=tf.train.batch([example,label],batch_size=9,num_threads=1,capacity=9)

# example_batch,label_batch=csvread(flielist)

In [7]:
# 开启会话运行结构
with tf.Session() as sess:
    # 定义一个线程协调器
    coord=tf.train.Coordinator()

    # 开启读取文件的线程
    threads=tf.train.start_queue_runners(sess,coord=coord)

    # 打印读取内容
    print(sess.run([example_batch,label_batch]))

    # 回收子线程
    coord.request_stop()
    coord.join(threads)

[array([b'row_id', b'0', b'1', b'2', b'3', b'4', b'5', b'6', b'7'],
      dtype=object), array([b'place_id', b'3073560757 9004412889 5652080691',
       b'1652178628 4379515211 6612350960',
       b'4894407065 3920195083 7608574746',
       b'7272466660 2004687925 1455486822',
       b'4720452725 4967325204 4782917866',
       b'1283939222 8208358948 2070306016',
       b'6021764482 7803008413 1644190109',
       b'7202788043 1649564607 2246348076'], dtype=object)]
